# TreeNode

In [3]:
public class TreeNode {
  int val;
  TreeNode left;
  TreeNode right;
  TreeNode() {}
  TreeNode(int val) { this.val = val; }
  TreeNode(int val, TreeNode left, TreeNode right) {
      this.val = val;
      this.left = left;
      this.right = right;
  }
}

### 难题 
[095: Unique-Binary-Search-Trees-II](https://leetcode.com/problems/unique-binary-search-trees-ii/)   真的难... 它还是分治    
[098: Validate-Binary-Search-Tree](https://leetcode.com/problems/validate-binary-search-tree/) 搜索树直接 InOrder traversal 看看是不是按大小就行了。

[108: Convert-Sorted-Array-to-Binary-Search-Tree](https://leetcode.com/problems/convert-sorted-array-to-binary-search-tree/)    用二分法结合

# Tree Traversal 

Tree Traversal 先考虑递归的 DFS 三种，用来命名。   
由于递归和Stack的相似性，可以延伸到类似图遍历的DFS和BFS，并且公用一个模板。     
再考虑类似 Morris Traversal 的用栈 DFS。 
最后是 O(1) Morris Traversal。      

## Basic BFS

In [25]:
// 以下这种BFS不考虑是否有null,如果有null直接移掉，非常暴力
public void BFS(TreeNode root, int k) {
    Queue < TreeNode > queue = new LinkedList();
    queue.add(root);
    while (!queue.isEmpty()) {
        if (queue.peek() != null) {
            TreeNode node = queue.remove();
            queue.add(node.left);
            queue.add(node.right);
        } else {
            queue.remove();
        } 
    }
}


## Basics DFS

### Inorder Recursive and Iterative

Iterative 核心是什么？通过左边一侧下到底，一路加到stack里面，然后pop出来顶部，然后操作，然后进入它的right节点，这就是逆时针的 inorder。而顺时针的 inorder 只需要把左右调换一下就可以了。为什么自然想到用 stack？ 因为DFS本身可以写递归，而递归的操作本身就是一个天然的stack结构。

In [3]:
//  逆时针 inorder
public List<Integer> inorderTraversal(TreeNode root) {
    List<Integer> values = new LinkedList<>();
    Stack<TreeNode> stack = new Stack<>();
    TreeNode cur = root;
    while( cur!= null || !stack.isEmpty() ) {
        while(cur!=null) {
            stack.push(cur);
            cur = cur.left;
        }
        cur = stack.pop();
        // 开始操作
        values.add(cur.val);
        // 结束操作
        cur = cur.right;
    }
    return values;
}

//  顺时针 inorder
public List<Integer> inorderTraversal(TreeNode root) {
    List<Integer> values = new LinkedList<>();
    Stack<TreeNode> stack = new Stack<>();
    TreeNode cur = root;
    while( cur!= null || !stack.isEmpty() ) {
        while(cur!=null) {
            stack.push(cur);
            cur = cur.right;
        }
        cur = stack.pop();
        // 开始操作
        values.add(cur.val);
        // 结束操作
        cur = cur.left;
    }
    return values;
}

### PreOrder and PostOrder
PreOrder 和 PostOrder 是一样的，这就是套用的一般的树图

In [5]:
// 逆时针 PreOrder，注意是先进行操作，再让节点入stack, 并且是右侧节点先入stack 
// 顺时针 PreOrder，则是左侧节点先入stack
public List<Integer> preorderTraversal(TreeNode root) {
    Stack<TreeNode> stack = new Stack<>();
    List<Integer> values = new LinkedList<>();

    if(root == null ) {
        return values;
    }        
    stack.push(root);
    while(!stack.isEmpty()) {
        TreeNode cur = stack.pop();
        values.add(cur.val);
        if( cur.right != null ) {
            stack.push(cur.right);
        }

        if( cur.left != null ) {
            stack.push(cur.left);
        }
    }
    return values;
}
// PostOrder只需要将PreOrder的结果逆序。
// 顺时针 PostOrder 的结果是 逆时针 PreOrder 逆序，逆序我们用 LinkedList.addFirst();

实际操作中，我觉得下面的写法更好，     
因为首先和 inOrder 变化不大。     
其次，上面的写法中，必须要防止root是null.     
最后这种写法很适合理解 Morris Traversal.    
当然了，上面的写法一改就是BFS了。

In [19]:
public class Solution {
    public List<Integer> preAndInOrderTraversal(TreeNode root) {
        Stack<TreeNode> stack = new Stack<>();
        List<Integer> preOrder = new ArrayList<>();
        List<Integer> inOrder  = new ArrayList<>();
        TreeNode cur = root;
        while(!stack.isEmpty() || cur != null){
            while(cur != null){
            
                preOrder.add(cur.val); // preOrder;
                
                stack.push(cur);
                
                cur = cur.left;
            }
            cur = stack.pop();
            
            inOrder.add(cur.val);   // inOrder
            
            cur = cur.right;
        }
        return inOrder;
    }
}

## BFS

In [24]:
public List<List<Integer>> levelOrder(TreeNode root) {
    List<List<Integer>> ans = new ArrayList<>();
    Queue<TreeNode> queue = new LinkedList<>();
    if(root == null){
        return ans;
    }
    queue.offer(root);
    while(!queue.isEmpty()){
        int size = queue.size();
        List<Integer> levelOrder = new ArrayList<>();
        while(size > 0 ) {
            TreeNode cur = queue.poll();

            levelOrder.add(cur.val);

            if(cur.left != null) {
                queue.offer(cur.left);
            }
            if(cur.right != null) {
                queue.offer(cur.right);
            }
            size--;
        }
        ans.add(levelOrder);
    }
    return ans;
}


### Morris Traversal
需要两个指针，一个 cur = root, 一个 prev = null;

In [20]:
  public List<Integer> preorderTraversal(TreeNode root) {
    LinkedList<Integer> output = new LinkedList<>();

    TreeNode cur = root;
    TreeNode prev = null;
      
    // Morris 的 cur 实际上还是和DFS一样，这是一个逆时针的走法。
    // Morris 考虑 InOrder 一个节点 node 应当排在它的所有左子树节点之后，而它的前一位就是它的左子树的极右节点。
    // 因此我们将该极右节点的右侧指向cur,这样就能做到在遍历完左子树之后从 node 的右侧继续遍历。
    // 按照逆时针遍历的话，
    // cur 走的路线始终是沿着最左侧下到底，然后回退到 InOrder 的下一位，再次沿最左侧下到底。
      
    // 虽然是InOrder的思路，但是如何记录遍历，还是由我们说了算的。
    // preOrder 就是只要遇到 cur 没访问过的元素，就立刻加进来。
    // inOrder 是记录 cur 首次访问的最左侧的元素，然后记录 cur 撤退回下一个InOrder节点路上的元素。

    while (cur != null) {
    // 逆时针遍历就是不停的下到极左，。
    // 如果当前节点存在左子树，那么我们处理左子树，否则说明当前节点是一个极左节点。
      if (cur.left == null) {
         // cur.left == null, 不存在左子树，此时cur必然是极左节点
        // 这里的元素都是首次访问的极左元素。而 inOrder 记录的首先是它，preOrder记录所有首次访问的元素。
        // 所以我们不论是 inOrder 还是 preOrder 都要记录它的值。
        output.add(cur.val);
        // 如果它右侧是空的，说明它也是某个极右节点，一定已经被指向了那个InOrder的下一位。
        // 如果它右侧不空，那么我们就进入它。
        // 所以我们可以放心的指向它的右边。
        cur = cur.right;
      } else {
        // 首先cur存在左子树，那么我们用 prev 取当前节点的左子树
        // 然后寻找它左子树的极右节点。即使这个子树是被访问过的，
        // (prev.right != cur) 保证了让 prev 停留在左子树的最右节点。
        // 即不论该左侧子树有木有被访问过，如下代码都能让 prev 停留在 cur 的左子树的极右，即InOrder序的前一位。
        // 因此叫 prev.
        prev = cur.left;
        while ( (prev.right != null) && (prev.right != cur) ) {
          prev = prev.right;
        }
        
        //如果cur未曾访问过如今的 prev, 那么 prev 右侧一定指向如今的 cur.
        // 反之我们就让他指向如今的 cur.
        if(prev.right == null ) {
            // 对于未被访问过的左子树，即 prev.right == null; 让它指向cur。
             prev.right = cur;
            // prev == null 说明 cur 未曾访问过该左子树，如果需要 preOrder，就应当记录了。
            // output.add(cur.val);
            // cur 进入左子树
            cur = cur.left;
        }else {
            // 对于已经被访问过的左子树，cur不需要进入了，那么把prev的指针拿掉。
            prev.right = null;
            // cur 的左子树都被访问过了，那么进入右子树。
            // 这里是cur退出访问已经访问过的左子树，如果需要 inOrder，就应当记录了。
            // output.add(cur.val);
            cur = cur.right;
        }
      }
    }
    return output;
  }

In [21]:
  public List<Integer> MorrisPreAndInOrderTraversal(TreeNode root) {
    LinkedList<Integer> preOrder = new LinkedList<>();
    LinkedList<Integer> inOrder = new LinkedList<>();
      
    TreeNode cur = root;
    TreeNode prev = null;

    while (cur != null) {

      if (cur.left == null) {
          
        preOrder.add(cur.val);  // For all
        inOrder.add(cur.val);  // For all
          
        cur = cur.right;

      } else {

        prev = cur.left;
        while ( (prev.right != null) && (prev.right != cur) ) {
          prev = prev.right;
        }

        if(prev.right == null ) {

            prev.right = cur;

            preOrder.add(cur.val); // preOrder

            cur = cur.left;
        }else {

            prev.right = null;

            inOrder.add(cur.val); // inOrder
            
            cur = cur.right;
        }
      }
    }
//     return preOrder;
      return inOrder;
  }



